# Phenotype Generation for Power Evaluation

### Define functions for generation

In [ ]:
# Load packages

if (!require(seqminer)) install.packages(seqminer)
if (!require(data.table)) install.packages(data.table)
if (!require(tibble)) install.packages(tibble)
if (!require(dplyr)) install.packages(dplyr)
if (!require(Matrix)) install.packages(Matrix)
if (!require(sparsesvd)) install.packages(sparsesvd)

library(seqminer)
library(data.table)
library(tibble)
library(dplyr)
library(Matrix)
library(sparsesvd)

# Load PLINK file
load_plink <- function(plink_prefix) {
    plink_obj <- seqminer::openPlink(plink_prefix)
    marker_index <- seq(nrow(plink_obj$bim))
    sample_index <- seq(nrow(plink_obj$fam))
    plink_matrix <- seqminer::readPlinkToMatrixByIndex(plink_prefix, sample_index, marker_index)
    colnames(plink_matrix) <- plink_obj$bim$V2

    return(plink_matrix)
}

# Read Group file and split variants by functional annotations
read_groupfile <- function(groupfile_name, gene_name) {
    groupfile <- file(groupfile_name, "r")
    line <- 0
    var <- NULL
    anno <- NULL

    while (TRUE) {
        line <- line + 1
        marker_group_line <- readLines(groupfile, n = 1)

        if (length(marker_group_line) == 0) {
            break
        }

        marker_group_line_list <- strsplit(marker_group_line, split = c(" +", "\t"))[[1]]

        if (marker_group_line_list[1] == gene_name) {
            if (marker_group_line_list[2] == "var") {
                var <- marker_group_line_list
            } else {
                anno <- marker_group_line_list
            }
        }
    }

    lof_idx <- which(anno == "lof")
    mis_idx <- which(anno == "missense")
    syn_idx <- which(anno == "synonymous")

    lof_var <- var[lof_idx]
    mis_var <- var[mis_idx]
    syn_var <- var[syn_idx]

    out <- list(lof_var, mis_var, syn_var)
    return(out)
}


# Convert genotypes for missing / alt-first PLINK files
convert_missing <- function(plink_matrix, ref_first=FALSE) {
    if (ref_first == FALSE) {   # alt-first
        # Convert missing genotypes to reference-homozygotes
        plink_matrix[which(plink_matrix == -9)] <- 2
        # Flip genotypes
        plink_matrix <- 2 - plink_matrix
    } else {
        plink_matrix[which(plink_matrix == -9)] <- 0
    }

    return(plink_matrix)
}


# function for counting MAF of input genotype matrix (used in split_plink_matrix)
count_maf <- function(geno_mat) {
    mac_vec <- rep(0, ncol(geno_mat))
    for (i in seq_len(ncol(geno_mat))) {
        # n0 <- length(which(geno_mat[, i]) == 0)
        n1 <- length(which(geno_mat[, i] == 1))
        n2 <- length(which(geno_mat[, i] == 2))

        mac <- n1 + 2 * n2
        mac_vec[i] <- mac
    }
    maf_vec <- mac_vec / (2 * nrow(geno_mat))
    common_list <- which(maf_vec > 0.01)

    out <- list(mac_vec, common_list)
    return(out)
}


# Split genotype matrix by function annotation (lof / mis / syn)
split_plink_matrix <- function(plink_matrix_converted, lof_var, mis_var, syn_var, mac_threshold = 10) {
    lof_mat <- plink_matrix_converted[, var_by_func_anno[[1]], drop = F]
    mis_mat <- plink_matrix_converted[, var_by_func_anno[[2]], drop = F]
    syn_mat <- plink_matrix_converted[, var_by_func_anno[[3]], drop = F]

    # count MAF for each functional annotation
    mac_lof <- count_maf(lof_mat)[[1]]
    common_list_lof <- count_maf(lof_mat)[[2]]

    mac_mis <- count_maf(mis_mat)[[1]]
    common_list_mis <- count_maf(mis_mat)[[2]]

    mac_syn <- count_maf(syn_mat)[[1]]
    common_list_syn <- count_maf(syn_mat)[[2]]

    # collapsing columns with MAC < threshold
    collapse_lof <- which(mac_lof < mac_threshold)
    collapse_mis <- which(mac_mis < mac_threshold)
    collapse_syn <- which(mac_syn < mac_threshold)

    if(length(collapse_lof) > 0) {
        rowsum_lof <- rowSums(lof_mat[, collapse_lof, drop = F])
        rowsum_lof[which(rowsum_lof > 1)] <- 1
        lof_mat_collapsed <- cbind(lof_mat[, -c(collapse_lof, common_list_lof), drop = F], rowsum_lof)
    } else if ((length(common_list_lof) > 0)) {
        lof_mat_collapsed <- lof_mat[, -c(common_list_lof), drop = F]
    } else {
        lof_mat_collapsed <- lof_mat
    }

    if(length(collapse_mis) > 0) {
        rowsum_mis <- rowSums(mis_mat[, collapse_mis, drop = F])
        rowsum_mis[which(rowsum_mis > 1)] <- 1
        mis_mat_collapsed <- cbind(mis_mat[, -c(collapse_mis, common_list_mis), drop = F], rowsum_mis)
    } else if ((length(common_list_mis) > 0)) {
        mis_mat_collapsed <- mis_mat[, -c(common_list_mis), drop = F]
    } else {
        mis_mat_collapsed <- mis_mat
    }

    if(length(collapse_syn) > 0) {
        rowsum_syn <- rowSums(syn_mat[, collapse_syn, drop = F])
        rowsum_syn[which(rowsum_syn > 1)] <- 1
        syn_mat_collapsed <- cbind(syn_mat[, -c(collapse_syn, common_list_syn), drop = F], rowsum_syn)
    } else if ((length(common_list_syn) > 0)) {
        syn_mat_collapsed <- syn_mat[, -c(common_list_syn), drop = F]
    } else {
        syn_mat_collapsed <- syn_mat
    }

    out <- list(lof_mat_collapsed, mis_mat_collapsed, syn_mat_collapsed)
    return(out)
}

### Phenotype by common variants

In [ ]:
# Install packages

if (!require(seqminer)) install.packages(seqminer)
if (!require(data.table)) install.packages(data.table)

# Load libraries

setwd("/media/leelabsg-storage0/kisung/META-SAIGE/script")
library(seqminer)
library(data.table)
source("230710_functions.R")

plink_prefix <- paste0("/media/leelabsg-storage0/kisung/META-SAIGE/data/genotype/common/WES_common_merged")
plink_matrix <- load_plink(plink_prefix)
plink_matrix_converted <- convert_missing(plink_matrix, ref_first = FALSE)
print("PLINK file for common variants loaded")

# Generate covariates and betas
set.seed(0)
n_sim <- 100
n_sample <- nrow(plink_matrix_converted)

mean_vec <- colMeans(plink_matrix_converted)
plink_matrix_centered <- plink_matrix_converted - rep(mean_vec, rep.int(nrow(plink_matrix_converted), ncol(plink_matrix_converted)))

for (n in 1:n_sim) {
    n_common <- ncol(plink_matrix_centered)
    cov1 <- rbinom(n_sample, 1, 0.1)
    cov2 <- rnorm(n_sample, 0, 1)
    betas <- rnorm(n_common, 0, 1 / n_common)

    # Phenotype generation (Common variant part)
    pheno_common <- plink_matrix_centered %*% betas

    out <- cbind(rownames(pheno_common), pheno_common, cov1, cov2)
    colnames(out)[1] <- "IID"
    colnames(out)[2] <- "pheno_common"
    outname <- paste0("/media/leelabsg-storage0/kisung/META-SAIGE/data/phenotype/common_231005/pheno_sim", n, ".txt")

    write.table(out, outname, row.names=F, col.names=T, quote=F)
    print(paste("Simulation", n, "completed"))
}

### Phenotype by rare variants

In [ ]:
# Install packages

if (!require(seqminer)) install.packages(seqminer)
if (!require(data.table)) install.packages(data.table)

# Load libraries

setwd("/media/leelabsg-storage0/kisung/META-SAIGE/script")
library(seqminer)
library(data.table)
source("230710_functions.R")

# Generate covariates and betas
set.seed(0)
n_sim <- 100
groupfile_name <- "/media/leelabsg-storage0/kisung/dnanexus/group_files/UKBexome_all_chr.txt"
gene_list <- c("IGSF9B", "BRCA2", "APOB", "GPRIN1", "CFB", "DDR1", "GPSM3", "HLA-DRB1", "DBH", "IL33")
chr_list <- c(11, 13, 2, 5, 6, 6, 6, 6, 9, 9)

for (i in seq_len(length(gene_list))) {
    gene_name <- as.character(gene_list[i])
    chr <- chr_list[i]

    rare_plink_prefix <- paste0("/media/leelabsg-storage0/kisung/META-SAIGE/data/genotype/rare_231005/", chr, "_", gene_name)
    rare_plink_matrix <- load_plink(rare_plink_prefix)
    rare_plink_matrix_converted <- convert_missing(rare_plink_matrix, ref_first = FALSE)
    n_sample <- nrow(rare_plink_matrix_converted)

    # mat_by_func_anno = [G_lof, G_mis, G_syn]
    var_by_func_anno <- read_groupfile(groupfile_name, gene_name)
    mat_by_func_anno <- split_plink_matrix(rare_plink_matrix_converted,
        var_by_func_anno[[1]],
        var_by_func_anno[[2]],
        var_by_func_anno[[3]],
        mac_threshold = 1
    )

    # rare_plink_matrix_converted[, var_by_func_anno[[2]]]

    # Genotype matrix for rare_variants
    lof_mat <- mat_by_func_anno[[1]][, c(1:(ncol(mat_by_func_anno[[1]]) - 1))]
    mis_mat <- mat_by_func_anno[[2]][, c(1:(ncol(mat_by_func_anno[[2]]) - 1))]
    syn_mat <- mat_by_func_anno[[3]][, c(1:(ncol(mat_by_func_anno[[3]]) - 1))]

    # Proportion of causal variants
    ## Scenario : (LoF, mis, syn) = (20%, 10%, 2%), (30%, 10%, 2%)
    lof_causal_rates <- c(0.2, 0.3)
    mis_causal_rates <- c(0.1, 0.1)
    syn_causal_rates <- c(0.02, 0.02)

    # Absolute effect sizes
    ## Scenarios : (LoF, mis, syn) = (0.3 log MAF, 0.15 log MAF, 0.15 log MAF), (0.5 log MAF, 0.25 log MAF, 0.25 log MAF)
    lof_effect_sizes <- c(0.3, 0.5, 0.7)
    mis_effect_sizes <- c(0.15, 0.25, 0.35)
    syn_effect_sizes <- c(0.15, 0.25, 0.35)

    # Effect directions
    ## Scenarios : (LoF, mis, syn) = (100%, 80%, 50%), (100%, 100%, 100%)
    lof_direction <- c(1, 1)
    mis_direction <- c(0.8, 1)
    syn_direction <- c(0.5, 1)

    # MAC by functional annotation
    lof_MAC <- colSums(lof_mat)
    mis_MAC <- colSums(mis_mat)
    syn_MAC <- colSums(syn_mat)

    # Generate causal vector
    betas_lof <- rep(0, ncol(lof_mat))
    betas_mis <- rep(0, ncol(mis_mat))
    betas_syn <- rep(0, ncol(syn_mat))

    for (n in 1:n_sim) {
        for (ii in 1:2) {
            for (jj in 1:3) {
                for (kk in 1:2) {
                    # Generate beta for LoF
                    for (j in seq_len(ncol(lof_mat))) {
                        MAF <- lof_MAC[j] * 2 / n_sample
                        if (lof_MAC[j] <= 10) {
                            betas_lof[j] <- rbinom(1, 1, 3 * lof_causal_rates[ii]) * abs(lof_effect_sizes[jj] * log(MAF, base = 10))
                        } else {
                            betas_lof[j] <- rbinom(1, 1, lof_causal_rates[ii]) * abs(lof_effect_sizes[jj] * log(MAF, base = 10))
                        }
                    }

                    # Generate beta for mis
                    for (j in seq_len(ncol(mis_mat))) {
                        MAF <- mis_MAC[j] * 2 / n_sample
                        if (mis_MAC[j] <= 10) {
                            betas_mis[j] <- rbinom(1, 1, 3 * mis_causal_rates[ii]) * abs(mis_effect_sizes[jj] * log(MAF, base = 10)) * (2 * (rbinom(1, 1, mis_direction[kk]) - 0.5))
                        } else {
                            betas_mis[j] <- rbinom(1, 1, mis_causal_rates[ii]) * abs(mis_effect_sizes[jj] * log(MAF, base = 10)) * (2 * (rbinom(1, 1, mis_direction[kk]) - 0.5))
                        }
                    }

                    # Generate beta for syn
                    for (j in seq_len(ncol(syn_mat))) {
                        MAF <- syn_MAC[j] * 2 / n_sample
                        if (syn_MAC[j] <= 10) {
                            betas_syn[j] <- rbinom(1, 1, 3 * syn_causal_rates[ii]) * abs(syn_effect_sizes[jj] * log(MAF, base = 10)) * (2 * (rbinom(1, 1, syn_direction[kk]) - 0.5))
                        } else {
                            betas_syn[j] <- rbinom(1, 1, syn_causal_rates[ii]) * abs(syn_effect_sizes[jj] * log(MAF, base = 10)) * (2 * (rbinom(1, 1, syn_direction[kk]) - 0.5))
                        }
                    }

                    # Phenotype generation (rare-variant part)
                    pheno_lof <- lof_mat %*% betas_lof
                    pheno_mis <- mis_mat %*% betas_mis
                    pheno_syn <- syn_mat %*% betas_syn

                    pheno_rv <- pheno_lof + pheno_mis + pheno_syn

                    # betas_lof_name <- paste0("/media/leelabsg-storage0/kisung/META-SAIGE/data/phenotype/rare/betas_lof_sim", n, "_gene", i, "_", ii, "_", jj, "_", kk, ".txt")
                    # betas_mis_name <- paste0("/media/leelabsg-storage0/kisung/META-SAIGE/data/phenotype/rare/betas_mis_sim", n, "_gene", i, "_", ii, "_", jj, "_", kk, ".txt")
                    # betas_syn_name <- paste0("/media/leelabsg-storage0/kisung/META-SAIGE/data/phenotype/rare/betas_syn_sim", n, "_gene", i, "_", ii, "_", jj, "_", kk, ".txt")

                    out <- cbind(rownames(pheno_rv), pheno_rv)
                    colnames(out) <- c("IID", "pheno_rv")

                    outname <- paste0("/media/leelabsg-storage0/kisung/META-SAIGE/data/phenotype/rare_231005/pheno_rare_sim", n, "_gene", i, "_", ii, "_", jj, "_", kk, ".txt")

                    # write.table(betas_lof, betas_lof_name, row.names=F, col.names=F, quote=F)
                    # write.table(betas_mis, betas_mis_name, row.names=F, col.names=F, quote=F)
                    # write.table(betas_syn, betas_syn_name, row.names=F, col.names=F, quote=F)
                    write.table(out, outname, row.names=F, col.names=T, quote=F)
                }
            }
        }
    }
    print(paste0("Gene ", i, " completed"))
}

### Make binary phenotypes

In [ ]:
library(data.table)
library(dplyr)

setwd("/media/leelabsg-storage0/kisung/META-SAIGE/data/phenotype")

n_sim <- 100

f <- function(x, pheno, p) {
    mean(1 / (1 + exp(-(pheno + x)))) - p
}

for (n in 1:n_sim) {
    pheno_common_name <- paste0("common_231005/pheno_sim", n, ".txt")
    pheno_common <- fread(pheno_common_name)
    for (ii in 1:2) {
        for (jj in 1:3) {
            for (kk in 1:2) {
                for (j in 1:10) {
                    pheno_rare_name <- paste0("rare_231005/pheno_rare_sim", n, "_gene", j, "_", ii, "_", jj, "_", kk, ".txt")
                    pheno_rare <- fread(pheno_rare_name)

                    if (j == 1) {
                        pheno_rare_total <- pheno_rare
                    } else {
                        pheno_rare_total$pheno_rv <- pheno_rare_total$pheno_rv + pheno_rare$pheno_rv
                    }
                }
                pheno <- left_join(pheno_common, pheno_rare_total, by="IID")
                pheno$pheno_total <- pheno$pheno_common + pheno$cov1 + pheno$cov2 + pheno$pheno_rv

                #prev <- c(0.01, 0.05, 0.1, 0.2)
                prev <- c(0.01, 0.05)
                for (pr in prev) {
                    a <- uniroot(f, c(-100, 100), pheno = pheno$pheno_total, p = pr)$root
                    prob <- 1 / (1 + exp(-(pheno$pheno_total + a)))

                    pheno_binary = rbinom(nrow(pheno), 1, prob)
                    pheno_out <- cbind(pheno, pheno_binary)
                    outname <- paste0("./total_231005/pheno_sim", n, "_", ii, "_", jj, "_", kk, "_prev", pr, ".txt")

                    write.table(pheno_out, outname, row.names=F, col.names=T, quote=F)
                }
            }
        }
    }
    print(paste0("Data for simulation ", n, " generated."))
}